In [1]:
import sys
import os

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Add the project root directory to the Python path
sys.path.append(project_root)

In [2]:
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Define the path to the data directory
data_dir = os.path.join(project_root, 'data')

# Example: Access a specific data file in the data directory
data_file_path = os.path.join(data_dir, 'RADCURE_challenge_clinical.csv')
df = pd.read_csv(data_file_path)

In [3]:
from AutoML.analyzer import Analyzer
from pprint import pprint

df.drop(columns=["Study ID", "split"], inplace=True)

config = Analyzer.dry_run(df)

pprint(config)

Config file not found. Creating custom...
Used a heuristic to define categorical and continuous columns. Please review!

Categorical: ['T Stage', 'Sex', 'Disease Site', 'Stage', 'N Stage', 'Dose', 'death', 'EGFRI', 'HPV Combined', 'Chemotherapy']
Continuous: ['age at dx', 'survival_time']
  - Outliers found in T Stage: ['T2 (2): 1 out of 2552', 'TX: 1 out of 2552', 'T3 (2): 1 out of 2552']
╒══════════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                          │                   │ Missing   │ Overall     │
╞══════════════════════════╪═══════════════════╪═══════════╪═════════════╡
│ n                        │                   │           │ 2552        │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ age at dx, mean (SD)     │                   │ 0         │ 62.0 (11.7) │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ survival_time, mean (SD) │                   │ 0         │ 4.8 (2.8)   │
├──────

In [4]:
import yaml

config['columns']['categorical'].remove('Dose')
config['columns']['continuous'].append('Dose') 

pprint(config)

with open('radcure_outputs/analyzer/config.yaml', 'w') as f:
            yaml.dump(config, f)

{'columns': {'categorical': ['T Stage',
                             'Sex',
                             'Disease Site',
                             'Stage',
                             'N Stage',
                             'death',
                             'EGFRI',
                             'HPV Combined',
                             'Chemotherapy'],
             'continuous': ['age at dx', 'survival_time', 'Dose'],
             'date': [],
             'other': []},
 'mapping': {'T Stage': {'T1': 'T1',
                         'T1a': 'T1a',
                         'T1b': 'T1b',
                         'T2': 'T2',
                         'T2 (2)': 'Other',
                         'T3': 'T3',
                         'T3 (2)': 'Other',
                         'T4': 'T4',
                         'T4a': 'T4a',
                         'T4b': 'T4b',
                         'TX': 'Other',
                         'Tis': 'Tis'}},
 'missingness_strategy': {'categorical': {

In [5]:
from AutoML.analyzer import Analyzer

analyzer = Analyzer(df, target_variable='death', output_dir='./radcure_outputs/analyzer', one_hot_encode=True, config='radcure_outputs/analyzer/config.yaml')

analyzer.run()

Applying changes from config...

╒══════════════════════════╤═══════════════════╤═══════════╤═════════════╕
│                          │                   │ Missing   │ Overall     │
╞══════════════════════════╪═══════════════════╪═══════════╪═════════════╡
│ n                        │                   │           │ 2552        │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ age at dx, mean (SD)     │                   │ 0         │ 62.0 (11.7) │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ survival_time, mean (SD) │                   │ 0         │ 4.8 (2.8)   │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ Dose, mean (SD)          │                   │ 0         │ 66.9 (5.6)  │
├──────────────────────────┼───────────────────┼───────────┼─────────────┤
│ T Stage, n (%)           │ Other             │           │ 3 (0.1)     │
├──────────────────────────┼───────────────────┼───────────┼───────

In [6]:
from AutoML.trainer import TrainerSupervised

df = pd.read_csv('./radcure_outputs/analyzer/updated_data.csv', index_col=0)
df.rename(columns={'survival_time': 'time', 'death':'event'}, inplace=True)

trainer = TrainerSupervised(task='time_to_event', output_dir='./radcure_outputs/ED_trainer_explainer',)
trainer.run(df, ['event','time'])

Training MTLR...
Training DeepHit...
Training DeepSurv...
Training CoxPH...
Training GradientBoosting...
Training RandomForest...
Training SVM...

Consolidated C-index Scores:
MTLR: 0.9531
DeepHit: 0.6640
DeepSurv: 0.6186
CoxPH: 0.7787
GradientBoosting: 0.7773
RandomForest: 0.7690
SVM: 0.7731


In [7]:
from AutoML.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <pycox.models.mtlr.MTLR object at 0x702aaeda9f50> does not.